# Periodic Life Table / Life Expectancy Calculator
This code calculates various estimates for the periodic life table and life expectancy including nax - the average number of person-years lived in the interval x to x + n - with iterative methods. The only data you need is age-specific death rates of 21 age groups, which means 0, 1-4, 5-9, ..., 85-89, 90-94, 95+. A little bit of modification will allow you to apply the code to different age group distributions. 

The methods to estimate nax are suggested by Keyfitz(1996) and Coale & Demeny(1983), according to **Demography** (Samuel H. Preston, Patrick Heuveline, Michel Guillot, 2001). nmx of this example is partly from 49 page of the same book. 

In [211]:
import numpy as np
import pandas as pd

AGEGROUPS = ['0', '1-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', 
             '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', 
             '90-94', '95+']
AGEGROUP_YEARS = [1, 4] + [5] * 19
NUM_AGEGROUPS = 21

In [212]:
dat = pd.read_csv("nmx_example.csv")
dat

,Age,nmx
0,0,0.008743
1,1-4,0.000370
2,5-9,0.000153
3,10-14,0.000193
4,15-19,0.000976
5,20-24,0.001285
6,25-29,0.001135
7,30-34,0.001360
8,35-39,0.001882
9,40-44,0.002935


In [213]:
def estimate_nax(nmx_input, woman): # woman: 0 = men, 1 = women
    
    nax = np.array([0]*2 + [2.5]*19)
    
    # first, estimate 1a0
    if (woman == 0): # for men
        if (nmx_input[0] >= 0.107):
            nax[0] = 0.330
        else:
            nax[0] = 0.045 + 2.684*nmx_input[0]
    else: # for women
        if (nmx_input[0] >= 0.107):
            nax[0] = 0.350
        else:
            nax[0] = 0.053 + 2.800*nmx_input[0]
    
    # next estimate 4a1
    if (woman == 0): # for men
        if (nmx_input[0] >= 0.107):
            nax[1] = 1.352
        else:
            nax[1] = 1.651 - 2.816*nmx_input[0]
    else: # for women
        if (nmx_input[0] >= 0.107):
            nax[1] = 1.361
        else:
            nax[1] = 1.522 - 1.518*nmx_input[0]
    
    # now we estimate the rest with 3 iterations
    
    for k in range(3):
        # update nqx and lx
        nqx = (AGEGROUP_YEARS * nmx_input) / (1 + (AGEGROUP_YEARS - nax) * nmx_input)
        nqx[20] = 1
        lx = np.array([100000] + [0] * 20)
        for i in range(1, len(nmx_input)):
            lx[i] = lx[i-1] * (1 - nqx[i-1])
        ndx = lx * nqx

    
        # update the rest of nax; nax[2] uses the weighted average of nax[0] and nax[1]
        nax[2] = 2.5 + (5/24)*((ndx[3] - (ndx[0]*(1/5) + ndx[1]*(4/5)))/ndx[2])
      
        for i in range(3, len(nmx_input)-1):
            nax[i] = 2.5 + (5/24)*((ndx[i+1] - ndx[i-1])/ndx[i])
    
    
    nax[len(nax)-1] = np.nan
    #return (nax[:20])
    return (nax) # you don't need the last entry as npx = 1 for the last age group


In [217]:
estimate_nax(dat['nmx'], 0)

array([ 0.06846621,  1.62637971,  1.96243591,  3.38526295,  2.73137451,
        2.52373509,  2.51081224,  2.61108666,  2.66935157,  2.70247594,
        2.6690006 ,  2.66861256,  2.68165006,  2.64720774,  2.60176876,
        2.60467779,  2.54979476,  2.46066323,  2.33336942,  2.08886408,
               nan])

In [222]:
def calculate_life_expectancy(nmx_input, woman):
    
    
    nax = np.array(estimate_nax(nmx_input, woman))
    nqx = np.array((AGEGROUP_YEARS * nmx_input) / (1 + (AGEGROUP_YEARS - nax) * nmx_input))
    nqx[NUM_AGEGROUPS-1] = 1
    npx = 1 - nqx
    
    lx = np.array([100000] + [0] * (NUM_AGEGROUPS-1))
    for i in range(1, len(nmx_input)):
        lx[i] = lx[i-1] * (npx[i-1])
    ndx = lx * nqx    
    
    nLx = np.array([0] * NUM_AGEGROUPS)
    Tx = np.array([0] * NUM_AGEGROUPS)
    nLx[NUM_AGEGROUPS-1] = lx[NUM_AGEGROUPS-1] / nmx_input[NUM_AGEGROUPS-1]
    Tx[NUM_AGEGROUPS-1] = nLx[NUM_AGEGROUPS-1]
    for i in range(NUM_AGEGROUPS-2, -1, -1):
        nLx[i] = (lx[i+1] * AGEGROUP_YEARS[i]) + (nax[i] * ndx[i])    
        Tx[i] = Tx[i+1] + nLx[i]
    
    ex = np.round(Tx / lx, 3)

    #life_TAB
    life_table = pd.DataFrame(
    {'nmx': np.array(nmx_input),'nax': nax, 'nqx': nqx, 'npx': npx, 'lx': lx, 
     'ndx': ndx, 'nLx': nLx, 'Tx': Tx, 'ex': ex}, index = AGEGROUPS, 
    columns = ['nmx', 'nax', 'nqx', 'npx', 'lx', 'ndx', 'nLx', 'Tx', 'ex'])
    
    return(life_table)

In [223]:
calculate_life_expectancy(dat['nmx'], 0)

,nmx,nax,nqx,npx,lx,ndx,nLx,Tx,ex
0,0.008743,0.068466,0.008672,0.991328,100000,867.236876,99191,7276472,72.765
1-4,0.000370,1.626380,0.001479,0.998521,99132,146.586622,396178,7177281,72.401
5-9,0.000153,1.962436,0.000765,0.999235,98985,75.688349,494693,6781103,68.506
10-14,0.000193,3.385263,0.000965,0.999035,98909,95.417449,494388,6286410,63.558
15-19,0.000976,2.731375,0.004869,0.995131,98813,481.142106,492969,5792022,58.616
20-24,0.001285,2.523735,0.006405,0.993595,98331,629.772738,490094,5299053,53.890
25-29,0.001135,2.510812,0.005659,0.994341,97701,552.891131,487128,4808959,49.221
30-34,0.001360,2.611087,0.006778,0.993222,97148,658.467092,484164,4321831,44.487
35-39,0.001882,2.669352,0.009369,0.990631,96489,903.996309,480338,3837667,39.773
40-44,0.002935,2.702476,0.014577,0.985423,95585,1393.314431,474720,3357329,35.124
